<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/CKIP_WS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''

R-Forge SVN(R語言)
https://github.com/r-forge/tmcn

===============================================
GPU

Colab使用
https://zhuanlan.zhihu.com/p/141205106

用 Google Colab 協助開發日常
https://ithelp.ithome.com.tw/articles/10234527

===============================================
時間處理

如何用pandas提取指定时间段的数据
https://blog.csdn.net/qq_38412868/article/details/107445068

'''
#CkipTagger
#https://github.com/ckiplab/ckiptagger/wiki/Chinese-README


#清除所有變數
#%reset -f


#安裝套件
!pip install tidytext
!pip install -U ckiptagger[tf,gdown]


#載入套件
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER


import os
import re
import collections   #詞頻統計
import numpy
import time
import datetime
import pandas as pd

     |████████████████████████████████| 106 kB 4.4 MB/s 
  Created wheel for tidytext: filename=tidytext-0.0.1-py3-none-any.whl size=3894 sha256=c2c8b32009b6bdf0fc4d30f2993755b45910fd48ee652ff76c95dc7cf7cf1655
  Stored in directory: /root/.cache/pip/wheels/46/3e/22/58c2c48258386098001ea3ef7652b69b6050fbcaa6b7adb9d7
  Created wheel for siuba: filename=siuba-0.1.2-py3-none-any.whl size=126968 sha256=2e752f6e47a26ae2b216bff230e1c419e2095e108d43e32cd9a0fd980539068e
  Stored in directory: /root/.cache/pip/wheels/bd/bf/fd/acdd87b7fa898b5af85090aba6ec6f258f6b97227851cf0f6a
Successfully built tidytext siuba


In [ ]:
#下載模型
#data_utils.download_data_gdown('/content/drive/MyDrive/Colab Notebooks/')

Mounted at /content/drive


In [ ]:
#載入模型  GPU 版本

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#ws = WS("/content/drive/MyDrive/Colab Notebooks/data", disable_cuda=False)
#pos = POS("/content/drive/MyDrive/Colab Notebooks/data", disable_cuda=False)
#ner = NER("/content/drive/MyDrive/Colab Notebooks/data", disable_cuda=False)



#載入模型  CPU 版本

ws = WS("/content/drive/MyDrive/Colab Notebooks/data")  #斷詞
pos = POS("/content/drive/MyDrive/Colab Notebooks/data")  #詞性標記
ner = NER("/content/drive/MyDrive/Colab Notebooks/data")  #專有名詞辨識

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)


In [ ]:
text = '代工大廠廣達 (2382-TW) 董事長林百里今 (12) 日出席天下經濟論壇，面對元宇宙 (Metaverse) 發展，看好包含 Meta(FB-US)、微軟 (MSFT-US) 等平台帶動商機成長，但對整體社會有信任 (Trust)、安全法規 (Security)、共識 (Acceptability) 等三大挑戰，台廠在此擁許多優勢，不過還需加強關注使用者行為的改變 (Behavior change)。林百里表示，元宇宙是廣達成立 33 年來，迎來的新挑戰，回顧過去布局，1988 年發展行動筆電、2000 年投資雲端伺服器、2020 年擴大運算 (Computing) 布局，2022 年迎來元宇宙，正持續發展智慧製造、交通、醫療等 Smart X，也做元宇宙的工具。林百里強調，廣達是做工具的，觀察元宇宙所需的硬體設備，包含基礎建設 (Infrastructure)、網路 (Network)、邊緣 (Edge)、雲端 (Cloud)、機器人 (Robotics)、IoT、XR 等都是可發展項目。展望元宇宙，林百里指出，臉書改名 Meta、微軟也擴大平台生態，都有助元宇宙概念成長，樂觀看待現實與虛擬世界的互動將助推新商機應用發酵，像是虛擬偶像、數位分身、大型互動虛擬演出、虛擬時尚等，更舉例一幅數位圖檔竟以 19.5 億元天價賣出，為虛擬藝術史 NFT 創造新里程碑。林百里笑說，當初對比特幣、狗狗幣覺得很不可思議，現階段也不會考慮花 19.5 億元買數位畫作，可預期未來很多東西都將在虛擬網路發生，自己與團隊都持續關注元宇宙各項發展，但元宇宙對整體社會還有信任 (Trust)、安全法規 (Security)、共識 (Acceptability) 等挑戰。元宇宙也將讓產業出現新變化，林百里認為，科技應用、使用者行為、營運模式都將相互連動，代表 AI、HI、BI 三者同等重要，過去工程師只做筆電產品，朝輕薄短小、省電就好，現在則需要從使用者行為 (Behavior)、整體解決方案回頭進行產品設計，這正是台廠較缺乏的部分。以廣達為例，林百里分享，四年前各事業群全部學習 AI 硬體，隨著 AI 發展成熟，現在不管是做智慧醫療、智慧城市、智慧運輸等，各部門都從 Smart 應用回過頭設計產品，而不是單從零件看產品，思維逐步轉變從解決方案出發。'
ws_results = ws([text])
ws_results